# Testing some things

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import foxes
import foxes.variables as FV
import foxes.constants as FC

import sys
sys.path.append( '/home/sengers/code/python/')
from util import *

## Generate LUT with optimal yaw misalignment

In [ ]:
## Initialize
range_WDs = np.arange(240,301,1) ## define WDs
range_yawms =  np.arange(-30,31,1) ## define yawms
sdata = pd.DataFrame(index=range(len(range_WDs)*len(range_yawms)))
sdata[FV.WS] = 8.0
sdata[FV.WD] = np.sort(list(range_WDs)*len(range_yawms))
yawm = np.asarray([[k,0.0] for k in range_yawms]*len(range_WDs))

## states
states = foxes.input.states.StatesTable(
    data_source=sdata,
    output_vars=[FV.WS, FV.WD, FV.TI, FV.RHO],
    fixed_vars={FV.RHO: 1.225, FV.TI: 0.05},
)

## Build and execute
mbook = foxes.ModelBook()
mbook.turbine_types["NREL5"] = foxes.models.turbine_types.PCtFile(
    "NREL-5MW-D126-H90.csv"
)
mbook.turbine_models["set_yawm"] = foxes.models.turbine_models.SetFarmVars()
mbook.turbine_models["set_yawm"].add_var(FV.YAWM, yawm)

farm = foxes.WindFarm()
farm.add_turbine(
    foxes.Turbine(
        xy=[0.0, 0.0],
        turbine_models=["set_yawm", "yawm2yaw", "NREL5", "kTI_05"],
    )
)
farm.add_turbine(
    foxes.Turbine(
        xy=[1000.0, 0.0],
        turbine_models=["set_yawm", "yawm2yaw", "NREL5", "kTI_05"],
    )
)


algo = foxes.algorithms.Downwind(
    mbook,
    farm,
    states,
    rotor_model="centre",
    wake_models=["PorteAgel_linear", "CrespoHernandez_max"],
    wake_frame="yawed",
    partial_wakes_model="auto",
    chunks=None,
    verbosity=0,
)
farm_results = algo.calc_farm()

In [ ]:
## Results
farm_df = farm_results.to_dataframe()
df = farm_df[[FV.WD,FV.YAWM]]
df = df.query('turbine==0').droplevel(level=1)
tmp = farm_df[[FV.P]]
tmp =tmp.groupby(level=0).sum() ## sum over two turbines
df = pd.concat([df,tmp],axis=1)

## Select yaw angle with max power
new = pd.DataFrame(index=range(len(range_WDs)),columns=[FV.WD,FV.YAWM,FV.P])
new[FV.WD] = range_WDs
for i,wd in enumerate(range_WDs):
    tmp = df.query('@wd-0.1 < WD < @wd+0.1')
    new.iloc[i,:] = tmp.iloc[np.argmax(tmp[FV.P]),:]

In [ ]:
## plot
fig,ax = plt.subplots()
ax2=ax.twinx()
ax.plot(new[FV.WD],new[FV.YAWM],'k-')
ax2.plot(new[FV.WD],new[FV.P],'r-')
ax.set_xlabel('$\delta\ [^{\circ}]$')
ax.set_ylabel('$\phi\ [^{\circ}]$')
axes_fontsize(ax,13)
axes_fontsize(ax2,13)
plt.show()